# Covariant Shift (Credit Model)

In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import xgboost as xgb

import pickle
from sklearn.externals import joblib

In [2]:
# Read all data
# df_cc = pd.read_csv('prep_data/credit_summary.csv',index_col='ip_id')
df_sa = pd.read_csv('prep_data/SA_summary.csv',index_col='ip_id')
df_sa2 = pd.read_csv('prep_data/summary_sa.csv',index_col='ip_id')
df_cc = pd.read_csv('prep_data/credit_summary.csv',index_col='ip_id',usecols=range(0,69))
df_cc_test = pd.read_csv('prep_data/credit_summary_test.csv',index_col='ip_id',usecols=range(0,68))
df_demo = pd.read_csv('data/demo.csv',index_col='ip_id')
df_y_train = pd.read_csv('data/y_train.csv',index_col='ip_id')
df_y_test = pd.read_csv('data/y_test_index.csv',index_col='ip_id')

In [3]:
# df_cc.columns.tolist()

In [4]:
df_cc = df_cc.drop(['label'],axis=1)

In [5]:
df_cc.head()

,number_of_huge_payment,n_cat1,n_cat10,n_cat11,n_cat12,n_cat13,n_cat14,n_cat15,n_cat16,n_cat2,...,n_card2,n_card3,most_frequent_card,sum_amt_card1,sum_amt_card2,sum_amt_card3,max_sum_amt_card,avg_amt_card1,avg_amt_card2,avg_amt_card3
ip_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,4.0,1.0,7.0,1.0,0.0,9.0,8.0,13.0,9.0,...,25,15,n_card1,305683.0,114261.0,66478.0,sum_amt_card1,5270.396552,4570.440000,4431.866667
2,0.0,2.0,1.0,7.0,0.0,1.0,14.0,6.0,5.0,8.0,...,30,13,n_card1,163238.0,129303.0,42568.0,sum_amt_card1,4411.837838,4310.100000,3274.461538
3,0.0,13.0,0.0,11.0,0.0,3.0,13.0,5.0,8.0,5.0,...,34,16,n_card1,239214.0,188063.0,77641.0,sum_amt_card1,5695.571429,5531.264706,4852.562500
4,0.0,8.0,0.0,7.0,0.0,4.0,13.0,1.0,6.0,7.0,...,30,15,n_card1,241518.0,139565.0,74907.0,sum_amt_card1,5138.680851,4652.166667,4993.800000
5,0.0,10.0,0.0,13.0,0.0,2.0,14.0,1.0,8.0,5.0,...,36,13,n_card1,247088.0,229892.0,71848.0,sum_amt_card1,5371.478261,6385.888889,5526.769231


In [6]:
# df_cc_test_sm = df_cc_test.iloc[:,0:62]

In [7]:
# df_cc_test_sm.shape

### Merge data

In [8]:
df_cc_all = pd.concat([df_cc,df_cc_test],axis=0)

In [9]:
df_cc_all.head()

,number_of_huge_payment,n_cat1,n_cat10,n_cat11,n_cat12,n_cat13,n_cat14,n_cat15,n_cat16,n_cat2,...,n_card2,n_card3,most_frequent_card,sum_amt_card1,sum_amt_card2,sum_amt_card3,max_sum_amt_card,avg_amt_card1,avg_amt_card2,avg_amt_card3
ip_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,4.0,1.0,7.0,1.0,0.0,9.0,8.0,13.0,9.0,...,25.0,15.0,n_card1,305683.0,114261.0,66478.0,sum_amt_card1,5270.396552,4570.440000,4431.866667
2,0.0,2.0,1.0,7.0,0.0,1.0,14.0,6.0,5.0,8.0,...,30.0,13.0,n_card1,163238.0,129303.0,42568.0,sum_amt_card1,4411.837838,4310.100000,3274.461538
3,0.0,13.0,0.0,11.0,0.0,3.0,13.0,5.0,8.0,5.0,...,34.0,16.0,n_card1,239214.0,188063.0,77641.0,sum_amt_card1,5695.571429,5531.264706,4852.562500
4,0.0,8.0,0.0,7.0,0.0,4.0,13.0,1.0,6.0,7.0,...,30.0,15.0,n_card1,241518.0,139565.0,74907.0,sum_amt_card1,5138.680851,4652.166667,4993.800000
5,0.0,10.0,0.0,13.0,0.0,2.0,14.0,1.0,8.0,5.0,...,36.0,13.0,n_card1,247088.0,229892.0,71848.0,sum_amt_card1,5371.478261,6385.888889,5526.769231


In [10]:
df_demo.join(df_cc_all,how="inner").join(df_y_train,how="left").fillna(2.0).label.value_counts()

0.0    5947
2.0    1127
1.0     414
Name: label, dtype: int64

In [11]:
df = df_demo.join(df_sa,how="inner").join(df_sa2,how="inner")
df = df.join(df_cc_all,how="inner")

In [12]:
df.to_csv('all_data.csv')

In [13]:
df.head()

,brth_yr,act_strt_dt,no_of_dpnd_chl,cis_income,crn_bal,gnd_cd,mar_st_cd,ctf_tp_cd,ocp_cd,Average_Debit,...,n_card2,n_card3,most_frequent_card,sum_amt_card1,sum_amt_card2,sum_amt_card3,max_sum_amt_card,avg_amt_card1,avg_amt_card2,avg_amt_card3
ip_id,,,,,,,,,,,,,,,,,,,,,
1,1990.0,2011-12-11,0.0,45864.0,876428.0,2.0,2.0,3.0,9,4403.238095,...,25.0,15.0,n_card1,305683.0,114261.0,66478.0,sum_amt_card1,5270.396552,4570.440000,4431.866667
2,1986.0,2010-06-08,1.0,43355.0,540649.0,2.0,2.0,1.0,3,4415.000000,...,30.0,13.0,n_card1,163238.0,129303.0,42568.0,sum_amt_card1,4411.837838,4310.100000,3274.461538
3,1983.0,2013-12-06,0.0,48836.0,310503.0,2.0,2.0,2.0,12,4261.840000,...,34.0,16.0,n_card1,239214.0,188063.0,77641.0,sum_amt_card1,5695.571429,5531.264706,4852.562500
4,1984.0,2009-12-27,0.0,44397.0,34209.0,2.0,2.0,5.0,11,2035.833333,...,30.0,15.0,n_card1,241518.0,139565.0,74907.0,sum_amt_card1,5138.680851,4652.166667,4993.800000
5,1986.0,2006-09-01,1.0,43281.0,39963.0,2.0,3.0,6.0,9,5165.400000,...,36.0,13.0,n_card1,247088.0,229892.0,71848.0,sum_amt_card1,5371.478261,6385.888889,5526.769231


### One-hot 

In [14]:
# df.drop(['act_strt_dt'],axis=1,inplace=True)
# dummy = pd.get_dummies(df[df.select_dtypes('object').columns.tolist()])
# df = pd.concat([dummy,df[df.select_dtypes(['int64','float64']).columns.tolist()]],axis=1)

### Drop

In [15]:
df.drop(df.select_dtypes('object').columns.tolist(),axis=1,inplace=True)

In [16]:
df.head()

,brth_yr,no_of_dpnd_chl,cis_income,crn_bal,gnd_cd,mar_st_cd,ctf_tp_cd,ocp_cd,Average_Debit,Average_Credit,...,n_tran_per_month,n_card1,n_card2,n_card3,sum_amt_card1,sum_amt_card2,sum_amt_card3,avg_amt_card1,avg_amt_card2,avg_amt_card3
ip_id,,,,,,,,,,,,,,,,,,,,,
1,1990.0,0.0,45864.0,876428.0,2.0,2.0,3.0,9,4403.238095,5151.714286,...,7.000000,58.0,25.0,15.0,305683.0,114261.0,66478.0,5270.396552,4570.440000,4431.866667
2,1986.0,1.0,43355.0,540649.0,2.0,2.0,1.0,3,4415.000000,6994.526316,...,5.714286,37.0,30.0,13.0,163238.0,129303.0,42568.0,4411.837838,4310.100000,3274.461538
3,1983.0,0.0,48836.0,310503.0,2.0,2.0,2.0,12,4261.840000,5186.647059,...,6.571429,42.0,34.0,16.0,239214.0,188063.0,77641.0,5695.571429,5531.264706,4852.562500
4,1984.0,0.0,44397.0,34209.0,2.0,2.0,5.0,11,2035.833333,4173.812500,...,6.571429,47.0,30.0,15.0,241518.0,139565.0,74907.0,5138.680851,4652.166667,4993.800000
5,1986.0,1.0,43281.0,39963.0,2.0,3.0,6.0,9,5165.400000,5356.625000,...,6.785714,46.0,36.0,13.0,247088.0,229892.0,71848.0,5371.478261,6385.888889,5526.769231


In [17]:
# df_y_train.head()

In [18]:
# df.isnull().sum()

In [19]:
df.columns[df.isnull().sum() > 0]

Index(['Average_Credit', 'Count_Credit', 'Average_Ratio', 'Count_Ratio', 'CR',
       'ATM', 'Branch', 'CDM', 'CorporateConnect', 'EDC', 'KPlus'],
      dtype='object')

In [20]:
df.fillna(0,inplace=True)

In [21]:
df_train = df.join(df_y_train,how="inner").drop(['label'],axis=1)
df_train['is_train'] = 1

df_test = df.join(df_y_test,how="inner")
df_test['is_train'] = 0

In [22]:
df_all = pd.concat([df_train,df_test],axis=0)

In [23]:
df_all.head()

,brth_yr,no_of_dpnd_chl,cis_income,crn_bal,gnd_cd,mar_st_cd,ctf_tp_cd,ocp_cd,Average_Debit,Average_Credit,...,n_card1,n_card2,n_card3,sum_amt_card1,sum_amt_card2,sum_amt_card3,avg_amt_card1,avg_amt_card2,avg_amt_card3,is_train
ip_id,,,,,,,,,,,,,,,,,,,,,
1,1990.0,0.0,45864.0,876428.0,2.0,2.0,3.0,9,4403.238095,5151.714286,...,58.0,25.0,15.0,305683.0,114261.0,66478.0,5270.396552,4570.440000,4431.866667,1
2,1986.0,1.0,43355.0,540649.0,2.0,2.0,1.0,3,4415.000000,6994.526316,...,37.0,30.0,13.0,163238.0,129303.0,42568.0,4411.837838,4310.100000,3274.461538,1
3,1983.0,0.0,48836.0,310503.0,2.0,2.0,2.0,12,4261.840000,5186.647059,...,42.0,34.0,16.0,239214.0,188063.0,77641.0,5695.571429,5531.264706,4852.562500,1
4,1984.0,0.0,44397.0,34209.0,2.0,2.0,5.0,11,2035.833333,4173.812500,...,47.0,30.0,15.0,241518.0,139565.0,74907.0,5138.680851,4652.166667,4993.800000,1
5,1986.0,1.0,43281.0,39963.0,2.0,3.0,6.0,9,5165.400000,5356.625000,...,46.0,36.0,13.0,247088.0,229892.0,71848.0,5371.478261,6385.888889,5526.769231,1


In [24]:
df_all.is_train.value_counts()

1    6361
0    1046
Name: is_train, dtype: int64

In [25]:
# df_all.isnull().sum()

In [26]:
# df_all.drop('label',axis=1,inplace=True)
# df_all.fillna(0,inplace=True)

In [27]:
# df_all.columns.tolist()

In [28]:
# columns = df_all.columns.tolist()

# y = df_all['is_train'].values
# X = df_all.drop(['is_train'],axis=1).values
# X_train, X_val, y_train, y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=0)

In [29]:
# rfc = RandomForestClassifier(n_jobs=-1, max_depth=5, min_samples_leaf = 5,
#                             n_estimators=100)
# rfc.fit(X_train,y_train)
# res_prob = rfc.predict_proba(X_val)[:, 1]

# fpr, tpr, thresholds = metrics.roc_curve(y_test.ravel(), res_prob, pos_label=1)
# metrics.auc(fpr, tpr)

In [30]:
# df_all.drop([columns[rfc.feature_importances_.argmax()]],axis=1,inplace=True)

### Feature Selection

In [54]:
def feature_select(df_all, thres = 0.5):
    res = 1
    while res > thres:
        columns = df_all.columns.tolist()

        y = df_all['is_train'].values
        X = df_all.drop(['is_train'],axis=1).values
        X_train, X_val, y_train, y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=0)
        
        
        rfc = RandomForestClassifier(n_jobs=-1, max_depth=5, min_samples_leaf = 5,
                                    n_estimators=100)
        rfc.fit(X_train,y_train)
        res_prob = rfc.predict_proba(X_val)[:, 1]

        fpr, tpr, thresholds = metrics.roc_curve(y_test.ravel(), res_prob, pos_label=1)
        res = metrics.auc(fpr, tpr)
        
        if res > thres:
            df_all= df_all.drop([columns[rfc.feature_importances_.argmax()]],axis=1)
    
    print(res)
    return df_all

In [32]:
df_select = feature_select(df_all,0.7)

0.6015477886317594


In [43]:
df_select.head()

,no_of_dpnd_chl,gnd_cd,number_of_huge_payment,n_cat10,sum_amt_cat10,avg_amt_cat10,is_train
ip_id,,,,,,,
1,0.0,2.0,0.0,1.0,2875.0,2875.0,1
2,1.0,2.0,0.0,1.0,3004.0,3004.0,1
3,0.0,2.0,0.0,0.0,0.0,0.0,1
4,0.0,2.0,0.0,0.0,0.0,0.0,1
5,1.0,2.0,0.0,0.0,0.0,0.0,1


### Build a Credit model with RandomForest

In [44]:
df_train = df_select.join(df_y_train,how="inner").drop(['is_train'],axis=1)

In [45]:
y = df_train['label'].values
X = df_train.drop(['label'],axis=1).values
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2,stratify=y,random_state=0)

In [46]:
rfc = RandomForestClassifier(n_jobs=-1, max_depth=5, min_samples_leaf = 5,
                            n_estimators=100)
rfc.fit(X_train,y_train)
res = rfc.predict(X_val)

In [47]:
print(metrics.classification_report(y_val,res))

             precision    recall  f1-score   support

        0.0       0.95      1.00      0.97      1190
        1.0       0.95      0.23      0.37        83

avg / total       0.95      0.95      0.93      1273



In [48]:
res_prob = rfc.predict_proba(X_val)[:,1]
fpr, tpr, thresholds = metrics.roc_curve(y_val.ravel(), res_prob, pos_label=1)
metrics.auc(fpr, tpr)

0.7242836893793662

In [57]:
joblib.dump(rfc, 'model/credit_model_rfc_3.joblib') 

['model/credit_model_rfc_3.joblib']

In [59]:
df_train.columns

Index(['no_of_dpnd_chl', 'gnd_cd', 'number_of_huge_payment', 'n_cat10',
       'sum_amt_cat10', 'avg_amt_cat10', 'label'],
      dtype='object')

### Build a Credit model with XgBoost

In [49]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_val, label=y_val)

In [50]:
param = {'max_depth': 10, 'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic', 'subsample':0.5}
param['nthread'] = 4
param['eval_metric'] = 'auc'

evallist = [(dtest, 'eval'), (dtrain, 'train')]

In [51]:
num_round = 10
bst = xgb.train(param, dtrain, num_round, evallist)

[0]	eval-auc:0.678825	train-auc:0.67649
[1]	eval-auc:0.67842	train-auc:0.678001
[2]	eval-auc:0.678374	train-auc:0.678001
[3]	eval-auc:0.70891	train-auc:0.722123
[4]	eval-auc:0.702602	train-auc:0.721341
[5]	eval-auc:0.705776	train-auc:0.726793
[6]	eval-auc:0.729447	train-auc:0.753806
[7]	eval-auc:0.728177	train-auc:0.754642
[8]	eval-auc:0.726121	train-auc:0.753645
[9]	eval-auc:0.725154	train-auc:0.755404


In [58]:
bst.save_model('model/credit_xgb_3.model')